In [ ]:
# Basic libraries
import numpy as np
import pandas as pd
import os

# Image processing
import cv2
from keras.api import *
from keras.api.preprocessing import image
from keras.api.preprocessing import image_dataset_from_directory 

# Model building
import tensorflow as tf
from keras.api.models import Sequential
from keras.api.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.api.optimizers import Adam
from keras.api.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth set for GPUs.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs detected.")

In [ ]:
# Path to the CSV file
data_path = "dataset/fer2013/fer2013/fer2013.csv"  # Update this path to your CSV file

# Load the dataset
df = pd.read_csv(data_path)

# Inspect the first few rows
df.head()

In [ ]:
# Convert pixel values from string to numpy array
def preprocess_images(df):
    images = []
    for pixel_sequence in df["pixels"]:
        img_array = np.array(pixel_sequence.split(), dtype="float32")
        img_array = img_array.reshape(48, 48, 1)  # Reshape to 48x48 and single channel
        img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
        images.append(img_array)
    return np.array(images)

# Preprocess images and extract labels
X = preprocess_images(df)
y = df["emotion"].values  # Emotion labels

# Check the shape of images and labels
print("Image data shape:", X.shape)  # Should be (num_samples, 48, 48, 1)
print("Label data shape:", y.shape)  # Should be (num_samples,)

In [ ]:
# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)

In [ ]:
# Define CNN model
model = Sequential([
    Conv2D(64, (3, 3), activation="relu", input_shape=(48, 48, 1), padding="same"),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.1),
    
    Conv2D(128, (5, 5), activation="relu", padding="same"),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.1),
    
    Conv2D(512, (3, 3), activation="relu", padding="same"),
    MaxPooling2D((2, 2)),
    Dropout(0.1),
    
    Conv2D(512, (3, 3), activation="relu", padding="same"),
    MaxPooling2D((2, 2)),
    Dropout(0.1),
    
    Flatten(),
    Dense(256, activation="relu"),
    BatchNormalization(),
    Dropout(0.1),
    
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.1),
    
    Dense(7, activation="softmax")
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.005),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

In [ ]:
# Define callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("model/emotions.keras", save_best_only=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint]
)